# Astropy Quantities

In astronomy (and most sciences) we typically have to deal with quantities which have some dimension (i.e. unit). The quantity is completely represented by not only a number but also a unit to go alongwith it. When we consider the same quantity in a different unit system, the quantity itself will not change, although the number used to represent it does change. While solving involved problems with several dimensional quantities, it may become cumbersome to keep track of the quantity if our program stores it as a single number with implied unit. Astropy units library is a useful way to keep track of a dimensional quantity 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u 
#The first step as usual is to import the required packages

We can initialise a dimensional quantity as follows

In [2]:
q = 42.0 * u.m 
lengths = [1,2,3]*u.m

We can access the numerical and unit part of q as follows

In [3]:
print(q.value,q.unit) #Ofcourse, we can use q.value with any other command also

print(lengths.value,q.unit)
print(lengths[0].value , lengths[0].unit)


42.0 m
[1. 2. 3.] m
1.0 m


We can multiply or divide quantities having different units but not add/subtract. Ofcourse, quantities with same unit can be added as usual. Trying to add incompatible quantities will give unit conversion error

In [4]:
distance2 =45 * u.m
distance1 =5*u.m
time = 2 * u.s

In [5]:
speed = (distance2 - distance1)/time
print(speed)

20.0 m / s


We can also use units library to interconvert between different unit systems using the to() method. Note that after calling this method, the value stored in quantity.value and quantity.unit don't change. Another method for converting to cgs/SI is also shown below.

In [6]:
print(speed.to(u.km/u.hour))
print(speed.value)
print(speed.unit)
print((1*u.Joule).cgs)
print((10*u.cm).si)

72.0 km / h
20.0
m / s
10000000.0 erg
0.1 m


We can also define new suitable units as required.

In [12]:
mph = u.imperial.mile/u.hour #astropy.units.imperial is the module for fps units
print(speed.to(mph))

44.73872584108805 mi / h


A special type of unit in the astropy.units package is the dimensionless quantity. This can either be initialised directly or come out from units cancelling out.

In [8]:
refractive_index = 1.5 *u.dimensionless_unscaled
print(refractive_index.value,refractive_index.unit) #No unit is printed, as the quantity is dimensionless

distance_ratio1 = (1*u.m)/(2*u.m)
print(distance_ratio1.value,distance_ratio1.unit)#This is a true unscaled dimensionless quantity

distance_ratio2 = (1*u.km)/(2*u.m)
print(distance_ratio2.value,distance_ratio2.unit)#This is also dimensionless, but value is scaled

print(distance_ratio1.unit == refractive_index.unit)#both are dimensionless unscaled
print(distance_ratio1.unit == distance_ratio2.unit)#both are dimensionless, but one is scaled, the other is not

1.5 
0.5 
0.5 km / m
True
False


Ofcourse, astropy being 'astro'py, does support usual astronomical units like parsecs, solar masses, AU etc

In [9]:
r = 1*u.AU
d = 1*u.parsec
print(d/r)
print(float(d/r)) #Notice how the float method is used to convert dimensionless scaled quatities to numbers
                  #The float method will not work with quantities having some dimension

AvgDensity_of_sun = 1*u.Msun/(4*np.pi/3*(1*u.Rsun)**3)
print(AvgDensity_of_sun.to(u.kg/u.m**3)) #Side note: note how this quite close to density of water

1.0 pc / AU
206264.8062454803
1409.8262953659105 kg / m3


The last remark to make about astropy is that it can also do 'implicit' conversions, like converting wavelength of light to frequency (in vacuum), by specifying the equivalencies parameter

In [10]:
print((1000 * u.nm).to(u.Hz, equivalencies=u.spectral()))  

299792457999999.94 Hz


# Colour in Astronomy
We see a variety of stars, whose colour we describe as 'red' or 'white' or 'blue'. But in astronomy, we are interested in a more quantitative definition of colour. But before we can come to this, we need to recall Planck's black body formula

Recall that Planck's formula for the black body spectrum is given by:
$$B(\lambda,t) = \frac{2hc^2}{\lambda^5}\frac{1}{e^{\frac{hc}{\lambda k_BT}}-1}$$